[ResNetをいろんな機械学習ライブラリで実装してみた~TensorFlow編~](https://qiita.com/gucchi0403/items/097d83f0e8a6091c1240)

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import re

In [4]:
def load_data():
    path_patch = '../../img/patch2/'
    names = !ls /Users/nakatani/labo/grabcut/img/foreground2_fixed
    no_s = [re.sub('.png', '', name) for name in names]
    all_data = []
    
    for no_ in no_s:
        path_img = path_patch + 'i' + no_ + '/'
        path_mask = path_patch + 'm' + no_ + '/'
        fnames = os.listdir(path_img)
        
        
        for fname in fnames:
            f_no = re.sub('.png', '', fname)
            img = cv2.imread(path_img + fname)
            mask = cv2.imread(path_mask + fname)
            label = 1 - int(f_no) % 2      #  blue が上 > 0     ,     green が上 > 1
            
            vec1d = np.hstack((img.flatten(), mask[:,:,:2].flatten(), label)) # 20 * 20 * 5 + 1 = 2001次元
            all_data.append(vec1d)
    all_data = np.array(all_data)
    return all_data

In [122]:
def split_data(data_set):
    
    np.random.shuffle(data_set)
    train = data_set[:2303]
    test = data_set[2303:]
    x_train_data = []
    y_train_data = []
    x_test_data = []
    y_test_data = []
    
    for data in train:
        rgb = np.reshape(data[:1200], (20,20,3))
        mask = np.reshape(data[1200:-1], (20,20,2))
        x_train_data.append(np.concatenate([rgb, mask], 2))
        y_train_data.append(data[-1])
        
    x_train_data = np.array(x_train_data)
    y_train_data = np.array(y_train_data)
    
    for data in test:
        rgb = np.reshape(data[:1200], (20,20,3))
        mask = np.reshape(data[1200:-1], (20,20,2))
        x_test_data.append(np.concatenate([rgb, mask], 2))
        y_test_data.append(data[-1])
        
    x_test_data = np.array(x_test_data)
    y_test_data = np.array(y_test_data)

    return x_train_data, y_train_data, x_test_data, y_test_data

In [123]:
all_data = load_data()

In [124]:
# num_img > 3290,    dimension > 2401
all_data.shape

(3290, 2001)

In [133]:
x_train_data, y_train_data, x_test_data, y_test_data = split_data(all_data)

In [134]:
import numpy as np
import tensorflow as tf

def weight_variable(shape, name=None):
    # 各層で使用する重み行列を返す関数
    # 標準偏差が0.1の切断正規分布からshapeで指定された形のテンソルを生成し、initialに代入
    initial = tf.truncated_normal(shape, stddev=0.1)

    # 初期のテンソルがinitialの変数tf.Variableを返す
    return tf.Variable(initial, name=name)

def softmax_layer(inpt, shape):
    # shapeで指定される形の重みをfc_wに代入
    fc_w = weight_variable(shape)

    # shape[1]で指定される形の重みをfc_bに代入
    #初期値はゼロベクトル
    fc_b = tf.Variable(tf.zeros([shape[1]]))

    # 全結合後、ソフトマックスを計算
    fc_h = tf.nn.softmax(tf.matmul(inpt, fc_w) + fc_b)

    return fc_h

def conv_layer(inpt, filter_shape, stride):
    # 入力データのチャンネル数をinpt_channelsに代入
    inpt_channels =  inpt.get_shape().as_list()[3]

    # Batch Normalization
    # チャンネル毎に平均meanと分散varを計算
    mean, var = tf.nn.moments(inpt, axes=[0,1,2])
    # Batch Normalizationに使用する学習パラメータbetaとgammaを準備
    # betaの初期値はゼロベクトル
    beta = tf.Variable(tf.zeros([inpt_channels]), name="beta")
    gamma = weight_variable([inpt_channels], name="gamma")
    # Batch Normalization実施
    batch_norm = tf.nn.batch_norm_with_global_normalization(
        inpt, mean, var, beta, gamma, 0.001,
        scale_after_normalization=True)

    # 活性化関数としてReLU関数使用
    out_relu = tf.nn.relu(batch_norm)

    # 畳み込み層
    # filter_shapeで指定される形の重みをfilter_に代入
    filter_ = weight_variable(filter_shape)
    # 畳み込み層の出力をoutに代入
    out = tf.nn.conv2d(out_relu, filter=filter_, strides=[1, stride, stride, 1], padding="SAME")

    return out

def residual_block(inpt, output_depth, stride=1, projection=False):
    # 入力データのチャンネル数をinput_depthに代入
    input_depth = inpt.get_shape().as_list()[3]

    # Batch Normalization + Relu +畳み込みを3セット
    conv1 = conv_layer(inpt, [1, 1, input_depth, int(output_depth/4)], stride)
    conv2 = conv_layer(conv1, [3, 3, int(output_depth/4), int(output_depth/4)], stride)
    conv3 = conv_layer(conv2, [1, 1, int(output_depth/4), output_depth], stride)

    # 入力と出力のチャンネル数が異なる場合は以下の2つの方法でチャンネル数を揃える
    if input_depth != output_depth:
        if projection:
            # Option B: Projection shortcut
            input_layer = conv_layer(inpt, [1, 1, input_depth, output_depth], 2)
        else:
            # Option A: Zero-padding
            # 足りない部分を0でパディング
            input_layer = tf.pad(inpt, [[0,0], [0,0], [0,0], [0, output_depth - input_depth]])
    else:
        input_layer = inpt

    # conv3に入力を足す
    res = conv3 + input_layer

    return res

ImportError: No module named 'tensorflow'

In [1]:
names = !ls /Users/nakatani/labo/grabcut/img/foreground2_fixed
path_w = './no.txt'
with open(path_w, mode='w') as f:
    f.write('\n'.join(names))